# Тестовое задание для Artec 3D
### Позиция PPC Specialist

> Необходимо проанализировать представленные данные на вкладке date и ответить на ряд вопросов:
> 1. в каком квартале показатель конверсии из клика в лид выше для стран Group B?
> 2. какая цена за лид в Q4 в кампаниях из Group A?
> 3. по какому из источников платного трафика суммарно больше всего лидов в Q3?
> 4. какой тип кампании имеет самый высокий показатель конверсии из клика в лид суммарно за два квартала?

In [167]:
# Ипортируем необходимые библиотеки
import pandas as pd
import numpy as np

In [168]:
# Считаем данные из xlsx файлов, отдельно сырые данные, отдельно страны
data = pd.read_excel("test.xlsx", sheet_name='data')
country = pd.read_excel("test.xlsx", sheet_name='country')

### Этап 0. Разведочный анализ данных
Посмотрим на типы данных, найдем аномалии, приведем типы

In [169]:
data.dtypes

Source                object
Country               object
Campaign              object
Year Month    datetime64[ns]
Impr.                float64
Clicks               float64
Cost                 float64
Leads                float64
dtype: object

* Year Month уже в удобном типе datetime64, но добавим еще и квартал отдельной колонкой
* Остальные поля в типе float64, посмотрим подробнее на них
* Как минимум Clicls и Leads должны быть целочисленными значениями

In [170]:
# Посмотрим на float64 поля
data.describe()

,Impr.,Clicks,Cost,Leads
count,20223.000000,21749.000000,21749.000000,1526.000000
mean,1502.642585,36.054564,31.626332,4.701990
std,7452.864768,170.893371,257.173608,6.794808
min,0.000000,0.000000,0.000000,2.234580
25%,6.000000,0.000000,0.000000,2.234580
50%,50.000000,1.345260,0.497851,2.234580
75%,476.000000,13.452600,5.884849,4.469160
max,317854.000000,8125.370400,16488.186367,127.371060


* Наблюдаю странную ситуацию, количество лидов и кликов могут быть нецелыми числами
* Кроме того, сумма расхода содержит после запятой больше двух знаков
* В реально жизни нужно запросить у автора выгрузки комментарий, почему входные данные не метчатся с ожиданиями
* Намеренно решаю не округлять, не приводить к иным типам

In [171]:
# Добавим квартал
data["quarter"] = data['Year Month'].dt.quarter

In [176]:
data.head(3)

,Source,Country,Campaign,Year Month,Impr.,Clicks,Cost,Leads,quarter
0,google,Anguilla,Search-brand-spanish,2021-07-01,1.0,0.0,0.0,0.0,3
1,google,Anguilla,Search-generic-eng-3st.group,2021-07-01,2.0,0.0,0.0,0.0,3
2,google,Antigua and Barbuda,Search-generic-eng-3st.group,2021-07-01,2.0,0.0,0.0,0.0,3


In [175]:
# Заменим все NaN значения нулям
data = data.fillna(0)

In [173]:
# Посмотрим на данные из листа со списком стран
country.dtypes

Group 1    object
Group 2    object
dtype: object

In [174]:
country.head(3)

,Group 1,Group 2
0,Australia,Georgia
1,Austria,Israel
2,Belgium,Cambodia


## Этап 1. Подготовим данные для анализа
* Разметим группой стран исходные данные, это нам потребуется для дальнейшей группировки

In [219]:
# Создадим list по группам
country_1 = country['Group 1'].tolist()
country_2 = country['Group 2'].tolist()

# Удалим пустые строки из листов
country_1 = [x for x in country_1 if str(x) != 'nan']
country_2 = [x for x in country_2 if str(x) != 'nan']

In [226]:
# Проверим, нет ли в сырых данных тех стран, которые отличаются от номенклатуры стран в листе country

country_raw = data["Country"].tolist()

# Оставим только уникальные значение в списке
country_raw = list(dict.fromkeys(country_raw))

In [239]:
# Посмотрим на пересечение списков
c = []
country_total = country_1 + country_2
 
for i in country_raw:
    if i in country_total:
        continue
    else:
        c.append(i)
 
print(c)

['South Korea', 'Austraia', 'Korea, South']


В данных для анализа обнаружено три страны, которых нету в справочнике. Размечу их вручную, в Group 1

In [240]:
# Добавим данные в список руками
country_1.append('South Korea')
country_1.append('Korea, South')
country_1.append('Austraia') # Скорее всего это Austria

In [242]:
# Разметим страны по группам

def country_setter(country):
    if country in country_1:
        return("Group_1")
    if country in country_2:
        return("Group_2")
    
# Применим фнукцию на датасет
data['Country_group'] = data.apply(lambda data: country_setter(data['Country']), axis=1)

In [245]:
# Посмотрим на первые строки таблицы, убедимся, что группа назначена
data.head(3)

,Source,Country,Campaign,Year Month,Impr.,Clicks,Cost,Leads,quarter,Country_group
0,google,Anguilla,Search-brand-spanish,2021-07-01,1.0,0.0,0.0,0.0,3,Group_2
1,google,Anguilla,Search-generic-eng-3st.group,2021-07-01,2.0,0.0,0.0,0.0,3,Group_2
2,google,Antigua and Barbuda,Search-generic-eng-3st.group,2021-07-01,2.0,0.0,0.0,0.0,3,Group_2


## Этап 2. Анализ данных

> 1. в каком квартале показатель конверсии из клика в лид выше для стран Group B?
> 2. какая цена за лид в Q4 в кампаниях из Group A?
> 3. по какому из источников платного трафика суммарно больше всего лидов в Q3?
> 4. какой тип кампании имеет самый высокий показатель конверсии из клика в лид суммарно за два квартала?

### 1. В каком квартале показатель конверсии из клика в лид выше для стран Group B?

In [250]:
# Скорее всего под группой стран Group B подразумевается Group 2 из исходных данных
# Сгруппируем по 'quarter','Country_group', посчитаем конверсию, найдем такой квартал, где показатель конверсии из клика в лид выше для Group 2

task_1 = data.groupby(['quarter','Country_group']).sum()
task_1['CR'] = task_1['Leads'] / task_1['Clicks']
task_1

Impr.        Clicks           Cost       Leads  \
quarter Country_group                                                        
3       Group_1         4934129.0  145694.34852  180906.224414  1678.16958   
        Group_2         7284335.0  168820.71318  196197.196951  1801.07148   
4       Group_1         7669684.0  201371.96940  157252.155605  1689.34248   
        Group_2        10499793.0  268263.67764  153485.517772  2006.65284   

                             CR  
quarter Country_group            
3       Group_1        0.011518  
        Group_2        0.010669  
4       Group_1        0.008389  
        Group_2        0.007480

#### Ответ: 
Формулировка задания неоднозначная, выше чем что?
* Выше, чем Group_1? — Таких кварталов нет, в каждом квартале, в Group_1 показатель конверсии выше
* Выше, чем Group_2, но в другом квартале? — Показатель конверсии в Group_2 максимален в квартале 3

### 2. Какая цена за лид в Q4 в кампаниях из Group A?

In [252]:
# Сгруппируем, посчитаем CPL
task_2 = data.groupby(['quarter','Country_group']).sum()
task_2['CPL'] = task_2['Cost'] / task_2['Leads']
task_2

Impr.        Clicks           Cost       Leads  \
quarter Country_group                                                        
3       Group_1         4934129.0  145694.34852  180906.224414  1678.16958   
        Group_2         7284335.0  168820.71318  196197.196951  1801.07148   
4       Group_1         7669684.0  201371.96940  157252.155605  1689.34248   
        Group_2        10499793.0  268263.67764  153485.517772  2006.65284   

                              CPL  
quarter Country_group              
3       Group_1        107.799728  
        Group_2        108.933598  
4       Group_1         93.084829  
        Group_2         76.488327

#### Ответ: цена за лид в Q4 в кампаниях из Group A = 93.08

### 3. По какому из источников платного трафика суммарно больше всего лидов в Q3?

In [254]:
# Сгруппируем по кварталу и источнику
task_3 = data.groupby(['quarter','Source']).sum()
task_3

Impr.        Clicks           Cost       Leads
quarter Source                                                          
3       bing          3392360.0   71475.00906   81807.405538   368.70570
        facebook       150052.0    3739.82280    1776.506120    44.69160
        facebook.com  2946333.0   86259.41646  132179.983531  1369.79754
        google        5729719.0  153040.81338  161339.526177  1696.04622
4       bing          4523266.0  132151.61610  105817.671701  1369.79754
        facebook       206245.0    5609.73420   10438.074745     8.93832
        facebook.com  5234587.0  134392.81926   93274.110758  1016.73390
        google        8205379.0  197481.47748  101207.816174  1300.52556

Допущу, что платный трафик это любой трафик, где заполнена Campaign, а значит в исходных данных не нужно разделять платный/бесплатный трафик. Каждая строчка в исходных данных имеет кампанию.

#### Ответ: больше всего лидов получено из канала google в Q3

###  4. Какой тип кампании имеет самый высокий показатель конверсии из клика в лид суммарно за два квартала?

* Допущу, что под типом кампании подразумевается префикс в названии, который заканчивается на _, либо на -, либо .
* Например в Search-brand-eng-rest.reg, тип это Search, а последующие слова подтип и название
* Таким образом у нас всего 4 типа кампаний: Search, Video, display, GDN

In [255]:
data.head(3)

,Source,Country,Campaign,Year Month,Impr.,Clicks,Cost,Leads,quarter,Country_group
0,google,Anguilla,Search-brand-spanish,2021-07-01,1.0,0.0,0.0,0.0,3,Group_2
1,google,Anguilla,Search-generic-eng-3st.group,2021-07-01,2.0,0.0,0.0,0.0,3,Group_2
2,google,Antigua and Barbuda,Search-generic-eng-3st.group,2021-07-01,2.0,0.0,0.0,0.0,3,Group_2


* Напишем костыль, который я бы не стал применять на проде, но для ad-hoc пойдет
* Если есть задача считать что-то в разрезе типа РК, то нужно выгружать этот тип

In [316]:
# Пишем функцию для разметки групп

def campaign_mapper(campaign):
    if campaign[0].lower() == "s":
        return('Search')
    if campaign[0].lower() == "v":
        return('Video')
    if campaign[0].lower() == "d":
        return("Display")
    if campaign[0].lower() == "g":
        return("GDN")

In [317]:
# Используем функцию
data['campaign_type'] = data.apply(lambda data: campaign_mapper(data['Campaign']), axis=1)

In [323]:
# Группируем по типу кампании и считаем конверсию
task_4 = data.groupby(['campaign_type']).sum()
task_4['CR'] = task_4['Leads'] / task_4['Clicks']
task_4

,Impr.,Clicks,Cost,Leads,quarter,CR
campaign_type,,,,,,
Display,8537217.0,230001.79272,237668.675153,2440.16136,24262,0.010609
GDN,1956041.0,46258.11036,27894.954450,484.90386,5726,0.010483
Search,19872284.0,507235.66404,421890.392941,4250.17116,46264,0.008379
Video,22399.0,655.14162,387.072198,0.00000,152,0.000000


#### Ответ: самый высокий показатель конверсии из клика в лид суммарно за два квартала имеет тип кампании Display